In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import os

In [2]:
# Read data

match_rep_df = pd.read_csv("checkpoints/match_rep_df.csv")


experiment_df = pd.read_csv("output/all-profile-metadata.csv")
experiment_df

,Vendor,Batch,Plate_Map_Name,Assay_Plate_Barcode,Modality,Images_per_well,Sites-SubSampled,Binning,Magnification,Number_of_channels,...,aperture,dry-immersion,vs-brightfield,simultaneous-excitation,sites,Size_MB,Size_MB_std,channel_names,brightfield_z_plane_used,feature_channels_found
0,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,4,NaN,1,10,6,...,0.45,dry,NaN,NaN,4,52.476334,0.000144,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA"
1,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,4,NaN,1,10,6,...,0.45,dry,NaN,NaN,4,52.476514,0.000183,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA"
2,MolDev,Scope1_MolDev_10X_4siteZ,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crestz,Confocal,4,NaN,1,10,6,...,0.45,dry,NaN,NaN,4,52.476631,0.000142,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA"
3,MolDev,Scope1_MolDev_20X_4site,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_20XPA_Crestz,Confocal,4,NaN,1,20,6,...,0.75,dry,NaN,NaN,4,52.476629,0.000114,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA"
4,MolDev,Scope1_MolDev_20X_9site,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_9site_20XPA,Confocal,9,NaN,1,20,6,...,0.75,dry,NaN,NaN,9,52.476334,0.000153,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_5Ch,JUMP-MOA_compound_platemap,BRO0117056_20x,Confocal,9,4.0,1,20,5,...,1.00,water,no,2.0,4,8.003585,0.000044,"AGP, DNA, ER, Mito, RNA",NaN,"AGP, DNA, ER, Mito, RNA"
180,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_5Ch_12Z,JUMP-MOA_compound_platemap,BRO0117056_20xb,Confocal,9,4.0,1,20,5,...,1.00,water,no,2.0,4,8.003585,0.000044,"AGP, DNA, ER, Mito, RNA",NaN,"AGP, DNA, ER, Mito, RNA"
181,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO0117059_20X,Confocal,9,4.0,1,20,6,...,1.00,water,yes,4.0,4,7.954439,0.000583,"AGP, BrightField, DNA, ER, Mito, RNA",Z08,"AGP, BrightField, DNA, ER, Mito, RNA"
182,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO01177034_20x,Confocal,9,4.0,1,20,6,...,1.00,water,yes,1.0,4,8.003584,0.000014,"AGP, BrightField, DNA, ER, Mito, RNA",Z17,"AGP, BrightField, DNA, ER, Mito, RNA"


For each profile, find the channel names from the load_data csvs. Then, find profiles that use 6-channel imaging and compare the actin to golgi correlation with 5 channel profiles

In [3]:
# Find channel names from load_data_csvs

def find_channel_names(metadata_df, load_data_path):
    plates = []
    df = metadata_df.copy()

    for root, dirs, files in os.walk(load_data_path):
        for file in files:
            if "load_data_with_illum.csv" in file:
                path = os.path.join(root, file)
                plate = os.path.split(path)[0].split("/")[-1]
                plates.append(plate)
                if plate not in metadata_df["Assay_Plate_Barcode"].values:
                    print(path, plate)
                columns = pd.read_csv(path).columns.tolist()
                channels = sorted([i.replace("FileName_Orig", "") for i in columns if "FileName_Orig" in i])
                channels = ", ".join(channels)
                # df.loc[df["Assay_Plate_Barcode"] == plate, "channel_names"] = ", ".join(channels)
                df.loc[df["Assay_Plate_Barcode"] == plate, "channel_names"] = channels
    return df

experiment_df = find_channel_names(experiment_df, "../jump-scope/load_data_csv/")
set(experiment_df.channel_names)

for i, _ in experiment_df.groupby("channel_names", sort=False, as_index=False):
    print(i)

# t.to_csv("test.csv")

../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate3_PCO_6ch_1site_10XPA_Crestz/load_data_with_illum.csv Plate3_PCO_6ch_1site_10XPA_Crestz
../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate3_PCO_6ch_1site_10XPA_Crest/load_data_with_illum.csv Plate3_PCO_6ch_1site_10XPA_Crest
../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate2-PCO-6ch-1site-10XPA/load_data_with_illum.csv Plate2-PCO-6ch-1site-10XPA
../jump-scope/load_data_csv/2020_11_16_Scope1_YokogawaUS/BRO0117014_20x/load_data_with_illum.csv BRO0117014_20x
Actin, DNA, ER, Golgi, Mito, RNA
AGP, DNA, ER, Mito
DNA, ER, Mito, RNA, WGPhalloidin
AGP, DNA, ER, Mito, RNA
AGP, BrightField, DNA, ER, Mito, RNA


In [4]:
for i, j in experiment_df.iterrows():
    print(j["channel_names"])

Actin, DNA, ER, Golgi, Mito, RNA
Actin, DNA, ER, Golgi, Mito, RNA
Actin, DNA, ER, Golgi, Mito, RNA
Actin, DNA, ER, Golgi, Mito, RNA
Actin, DNA, ER, Golgi, Mito, RNA
Actin, DNA, ER, Golgi, Mito, RNA
Actin, DNA, ER, Golgi, Mito, RNA
AGP, DNA, ER, Mito
AGP, DNA, ER, Mito
AGP, DNA, ER, Mito
AGP, DNA, ER, Mito
AGP, DNA, ER, Mito
AGP, DNA, ER, Mito
AGP, DNA, ER, Mito
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, Mito, RNA, WGPhalloidin
DNA, ER, 

In [53]:
def search_correlations(metadata_df, profile_path, aggregate_on=None):
    """
    Reads *_normalized_feature_select_negcon_batch.csv.gz
    and performs a function on the dataframe 
    """
    df_list = []
    # metadata_df = metadata_df[~metadata_df["Batch"].str.contains("siteSub")]
    df = metadata_df.copy()
    df = df[~df["Batch"].str.contains("siteSub")]
    for ind, row in metadata_df.iterrows():
        df_path = os.path.join(profile_path, row["Batch"], row["Assay_Plate_Barcode"], f"{row['Assay_Plate_Barcode']}.csv.gz")
        print(df_path)
        df_path = glob.glob(df_path)[0]
        load_df = pd.read_csv(df_path)
        # cols = [i for i in load_df.columns if ("Golgi" in i and "Actin" in i and "Costes" in i) or 
        #                                         ("Golgi" in i and "DNA" in i and "Costes" in i) or
        #                                         ("Actin" in i and "DNA" in i and "Costes" in i) or
        #                                         ("AGP" in i)
        #                                         ]
        cols = [i for i in load_df.columns if "Correlation" in i]
        # cols = [i for i in load_df.columns if i in ["Actin", "DNA", "ER", "Golgi", "Mito", "RNA"]]
        load_df = load_df[cols]
        # For the masked dataset (which represents one plate), aggregate all well values
        load_df.agg(np.mean)
        # Since the DF was masked by [cols], add back in the Batch, Assay_Plate_Barcode and channel_names, based on the row info
        load_df["Batch"], load_df["Assay_Plate_Barcode"], load_df["channel_names"] = [row["Batch"], row["Assay_Plate_Barcode"], row["channel_names"]]
        df_list.append(load_df)
    out_df = pd.concat(df_list)
    # Once concatenated, aggregate all values based on channel_name grouping
    if aggregate_on is not None:
        out_df = out_df.groupby(
                            aggregate_on,as_index=False).agg(np.mean)
    return out_df

corr_df = search_correlations(experiment_df, "../jump-scope/profiles/")
# search_correlations(experiment_df, "../jump-scope/profiles/")


../jump-scope/profiles/Scope1_MolDev_10X/Plate2_PCO_6ch_4site_10XPA/Plate2_PCO_6ch_4site_10XPA.csv.gz
../jump-scope/profiles/Scope1_MolDev_10X/Plate3_PCO_6ch_4site_10XPA_Crest/Plate3_PCO_6ch_4site_10XPA_Crest.csv.gz
../jump-scope/profiles/Scope1_MolDev_10X_4siteZ/Plate3_PCO_6ch_4site_10XPA_Crestz/Plate3_PCO_6ch_4site_10XPA_Crestz.csv.gz
../jump-scope/profiles/Scope1_MolDev_20X_4site/Plate3_PCO_6ch_4site_20XPA_Crestz/Plate3_PCO_6ch_4site_20XPA_Crestz.csv.gz
../jump-scope/profiles/Scope1_MolDev_20X_9site/Plate2_PCO_6ch_9site_20XPA/Plate2_PCO_6ch_9site_20XPA.csv.gz
../jump-scope/profiles/Scope1_MolDev_20X_9site/Plate3_PCO_6ch_9site_20XPA_Crest/Plate3_PCO_6ch_9site_20XPA_Crest.csv.gz
../jump-scope/profiles/Scope1_MolDev_20X_Adaptive/Plate3_PCO_6ch_Adaptive_20XPA/Plate3_PCO_6ch_Adaptive_20XPA.csv.gz
../jump-scope/profiles/Scope1_Nikon_10X/BR00117060a10x/BR00117060a10x.csv.gz
../jump-scope/profiles/Scope1_Nikon_10X/BR00117061a10x/BR00117061a10x.csv.gz
../jump-scope/profiles/Scope1_Nikon_10X/

In [49]:
# Which profiles do not have Costes correlation

for i, _df in corr_df.groupby(["Batch", "Assay_Plate_Barcode"],as_index=False):
    # print(len(_df.dropna()))
    if len(_df.dropna()) == 0:
        print(_df["Batch"].unique())
    else:
        print(_df["Batch"].unique(), "contains Costes")


['1siteSubSample_Scope1_MolDev_10X'] contains Costes
['1siteSubSample_Scope1_MolDev_10X'] contains Costes
['1siteSubSample_Scope1_MolDev_10X_4siteZ'] contains Costes
['1siteSubSample_Scope1_MolDev_20X_4site'] contains Costes
['1siteSubSample_Scope1_MolDev_20X_9site'] contains Costes
['1siteSubSample_Scope1_MolDev_20X_9site'] contains Costes
['1siteSubSample_Scope1_Nikon_20X']
['1siteSubSample_Scope1_Nikon_20X']
['1siteSubSample_Scope1_Nikon_20X']
['1siteSubSample_Scope1_PE_Bin1_Confocal_1Plane']
['1siteSubSample_Scope1_PE_Bin1_Confocal_1Plane']
['1siteSubSample_Scope1_PE_Bin1_Confocal_1Plane']
['1siteSubSample_Scope1_PE_Bin1_Confocal_1Plane']
['1siteSubSample_Scope1_PE_Bin1_Confocal_3Plane']
['1siteSubSample_Scope1_PE_Bin1_Confocal_3Plane']
['1siteSubSample_Scope1_PE_Bin1_Confocal_3Plane']
['1siteSubSample_Scope1_PE_Bin1_Confocal_3Plane']
['1siteSubSample_Scope1_PE_Bin1_Widefield_1Plane']
['1siteSubSample_Scope1_PE_Bin1_Widefield_1Plane']
['1siteSubSample_Scope1_PE_Bin1_Widefield_1Plan

In [27]:


out_df = corr_df.groupby(
                    ["Batch", "Assay_Plate_Barcode"],as_index=False).agg(np.mean)

out_df[
    (~out_df["Batch"].str.contains("siteSub"))
    & (out_df["Batch"].str.contains("Scope1_Yokogawa_US_40X_BRO0117059"))
    ].dropna(axis=1)

,Batch,Assay_Plate_Barcode
183,Scope1_Yokogawa_US_40X_BRO0117059,BRO0117059_40x


In [28]:
# Why does the Yokogawa profile not have costes correlations?

profile_path =  "../jump-scope/profiles/"

for ind, row in experiment_df[experiment_df["Batch"].str.contains("Scope1_Yokogawa_US_40X_BRO0117059")].iterrows():
        df_path = os.path.join(profile_path, row["Batch"], row["Assay_Plate_Barcode"], f"{row['Assay_Plate_Barcode']}.csv.gz")
        df_path = glob.glob(df_path)[0]
        load_df = pd.read_csv(df_path)

In [56]:
[i for i in corr_df.columns if "Correlation" in i]

['Cells_Correlation_Correlation_Actin_DNA',
 'Cells_Correlation_Correlation_Actin_ER',
 'Cells_Correlation_Correlation_Actin_Golgi',
 'Cells_Correlation_Correlation_Actin_Mito',
 'Cells_Correlation_Correlation_Actin_RNA',
 'Cells_Correlation_Correlation_DNA_ER',
 'Cells_Correlation_Correlation_DNA_Golgi',
 'Cells_Correlation_Correlation_DNA_Mito',
 'Cells_Correlation_Correlation_DNA_RNA',
 'Cells_Correlation_Correlation_ER_Golgi',
 'Cells_Correlation_Correlation_ER_Mito',
 'Cells_Correlation_Correlation_ER_RNA',
 'Cells_Correlation_Correlation_Golgi_Mito',
 'Cells_Correlation_Correlation_Golgi_RNA',
 'Cells_Correlation_Correlation_Mito_RNA',
 'Cells_Correlation_Costes_Actin_DNA',
 'Cells_Correlation_Costes_Actin_ER',
 'Cells_Correlation_Costes_Actin_Golgi',
 'Cells_Correlation_Costes_Actin_Mito',
 'Cells_Correlation_Costes_Actin_RNA',
 'Cells_Correlation_Costes_DNA_Actin',
 'Cells_Correlation_Costes_DNA_ER',
 'Cells_Correlation_Costes_DNA_Golgi',
 'Cells_Correlation_Costes_DNA_Mito',
 

In [57]:
# Find a subset of combinations
mask_cols = corr_df.columns[corr_df.columns.str.contains("Costes_Actin|channel_names")]
corr_df = corr_df[mask_cols]

plot_corr_df = corr_df[(corr_df["channel_names"] == "Actin, DNA, ER, Golgi, Mito, RNA")].dropna(axis=1).transpose().reset_index()
plot_corr_df

,index,3
0,channel_names,"Actin, DNA, ER, Golgi, Mito, RNA"
1,Cells_Correlation_Costes_Actin_DNA,0.34024
2,Cells_Correlation_Costes_Actin_ER,0.609428
3,Cells_Correlation_Costes_Actin_Golgi,0.82134
4,Cells_Correlation_Costes_Actin_Mito,0.563247
5,Cells_Correlation_Costes_Actin_RNA,0.710706


In [58]:
plot_corr_df.iloc[1:,:]

,index,3
1,Cells_Correlation_Costes_Actin_DNA,0.34024
2,Cells_Correlation_Costes_Actin_ER,0.609428
3,Cells_Correlation_Costes_Actin_Golgi,0.82134
4,Cells_Correlation_Costes_Actin_Mito,0.563247
5,Cells_Correlation_Costes_Actin_RNA,0.710706


In [59]:
plot_corr_df.iloc[1:,:].values

array([['Cells_Correlation_Costes_Actin_DNA', 0.3402400225698437],
       ['Cells_Correlation_Costes_Actin_ER', 0.6094276829056531],
       ['Cells_Correlation_Costes_Actin_Golgi', 0.8213403666698823],
       ['Cells_Correlation_Costes_Actin_Mito', 0.5632471927358672],
       ['Cells_Correlation_Costes_Actin_RNA', 0.7107060016399768]],
      dtype=object)

In [56]:
plot_corr_df.iloc[1:,:]

,3
Cells_Correlation_Costes_Actin_DNA,0.34024
Cells_Correlation_Costes_Actin_ER,0.609428
Cells_Correlation_Costes_Actin_Golgi,0.82134
Cells_Correlation_Costes_Actin_Mito,0.563247
Cells_Correlation_Costes_Actin_RNA,0.710706
